# Experimentos
En general se quiere probar que
1- Los algoritmos permiten aprender

2- Los algoritmos generalizan

3- Los algoritmos entrenan mas rápido que la wea cruda (Estabilidad, velocidad)

5- Cuanto depende del dataset ese aprendizaje

4- Son capaces de aprender de alguna distribución generada en la vecindad del dataset

6- Permiten una mejor estabilidad frente al ruido que el entrenamiento en crudo (En que momento pasa el umbral de tanto x)

7- Feats permiten una mejor estabilidad frente al ruido que el uso de logits solo

8- Funcionamiento de feats > funcionamiento de KD > funcionamiento de CE solo


Por cada experimento interesa

1- velocidad de convergencia (estabilidad?) (En que epoca el algoritmo pasa 3 epochs sin crecer)

2- Acc en test 

3- Acc en test / train

4- Mejores y peores

5- Delta sobre KD, CE

6- grafico que permita ver tendencia según tecnica/capa

7- media de feats por capa

Los experimentos disponibles sobre Cifar 10 son

| Experimento    | ce | kd | feat |   |
|----------------|----|----|------|---|
| Dataset Normal | x  | x  | x    |   |
| Ruido  (0,1-1) |    | x  | x    |   |
| GAN            |    | x  | x    |   |
| VAE            |    | x  | x    |   |

Podria faltar grid de ruido con temperatura
Podría ser interesante probar con alguna medida de distancia entre y_t y y_s, implica correr todo de nuevo

### Carga de datos

In [1]:
import pandas as pd
import altair as alt
import numpy as np

from lib.utils.viz.load_data import *
from lib.utils.viz.plot_utils import *
from lib.utils.viz.table_utils import *
from altair_saver import save

# Dataset Normal

* Dataset Cifar 10 sin modificaciones

El primer experimento consistió en una prueba de la tecnica sobre el dataset Cifar 10 sin ninguna modificacion. Se comenzó por entrenar 

**Valores Medidos**
Se consideran en general 3 valores interesantes que medir, 
* La diferencia del accuracy en validacion entre la red tutora y los modelos destilados: Mide cuanto logra aprender el método en la configuracion usada.
* La diferencia del accuracy en validacion entre el modelo entrenado en CE y los modelos destilados. Mide cuanto mejora el desempeño del modelo con respecto a su propia base usando el metodo.
* El ratio entre el accuracy en entrenamiento y en validacion del modelo destilado: Mide cuanto overfitting presenta el modelo.
* Epoca en que se alcanza el accuracy de validacion mas alto o la perdida mas baja: Permite dar una estimacion vaga de cuando el modelo logra converger.

**Experimentos realizados**
1. Se entrenó los modelos Resnet101, Resnet18 y MobileNet usando cross entropy (CE) sobre los datos. Esto permite tener una base desde la que comparar y un modelo tutor desde el que destilar.
2. Se destiló usando Resnet101 como tutor junto a Resnet18 y MobileNet como estudiantes. Se varió la temperatura entre 1 y 100 para estudiar cuanto afecta la temperatura al valor obtenido de destilacion. Esto se realizó usando KD solo y KD simultaneamente con Cross Entropy (KD_CE).
3. Se destiló usando distintas perdidas de destilacion de features sobre distintas capas. Esto permite evaluar el efecto del uso de features al momento de destilar. Con respecto a la perdida en los logits, se fijó la temperatura en 8, y se probó usando KD solo, KD_CD y CE solo


## Entrenamiento de redes tutoras

Lo primero que se hizo fue el entrenamiento del modelo a ser entrenado como red tutora y los modelos estudiantes usando el dataset sin ninguna modificación y CE como perdida por 400 epocas. Los datos medidos en esta fase se adjuntan en la siguiente tabla. 

In [2]:
#print(get_top(data['teachers'],add_fields=" 	loss_conv 	acc_conv 	test/train ".split(),
#        drop_fields="train_step 	test_step	delta_student 	student 	log_dist 	feat_dist 	layer".split()).to_latex())
get_top(data['teachers'],add_fields=" 	loss_conv 	acc_conv 	test/train ".split(),
        drop_fields="train_step 	test_step	delta_student 	student 	log_dist 	feat_dist 	layer".split())

,model,epoch,loss_conv,acc_conv,delta_teacher,test/train
model,,,,,,
ResNet101,ResNet101,400,339,391,0.0,0.940566
ResNet18,ResNet18,400,360,303,-0.6,0.934840
MobileNet,MobileNet,400,391,306,-4.0,0.903520


De la tabla es posible ver que se alcanzan en todos los casos valores relativamente altos y no tan distantes entre si de accuracy en la base de datos de entrenamiento. Ademas, es posible notar un buen nivel de ratio nivel bastante cercano a 1 en el ratio entre el accuracy de entrenamiento y validacion. 
Los valores minimos de perdida y maximo de accuracy se se obtuvieron durante el ultimo tramo de entrenamiento, sin embargo, tal y como es posible apreciar en el grafico siguiente estos valores se estabilizan bastante durante las primeras 100 epocas de entrenamiento. Al ver estos en tanto tiempo es posible notar que el tiempo de entrenamiento para ResNet 101 es unas 3 veces mayor que el tiempo de entrenamiento en los otros modelos.

In [3]:

teacher_data=dict([(k,epoch_data[i]) for k,i in list(zip(conv_data['teachers']['index'],conv_data['teachers']['t'])) ])
source=pd.concat([d2plambda(v,k) for k,v in teacher_data.items()])

x1='epochs'
x2='test/wall_time'
y='test/acc'
color='model'

g=alt.hconcat(
alt.Chart(source,title="Accuracy en entrenamiento contra epocas").mark_line().encode(
    alt.Y(y, type='quantitative',title=global_titles[y],scale=alt.Scale(zero=True,base=10,type='linear')),
    alt.X(x1, type='quantitative',title=global_titles[x1],scale=alt.Scale(zero=True,base=10,type='linear')),
    color=alt.Color('%s:N'%color, legend=alt.Legend(title=global_titles[color]), scale=alt.Scale(scheme='spectral')),
    #strokeDash='model',
).interactive()
,
alt.Chart(source,title="Accuracy en entrenamiento contra tiempo").mark_line().encode(
    alt.Y(y, type='quantitative',title=global_titles[y],scale=alt.Scale(zero=True,base=10,type='linear')),
    alt.X(x2, type='quantitative',title=global_titles[x2],scale=alt.Scale(zero=True,base=10,type='time',nice=True)),
    color=alt.Color('%s:N'%color, legend=alt.Legend(title=global_titles[color]), scale=alt.Scale(scheme='spectral')),
    #strokeDash='model',
).interactive()
)
g

alt.HConcatChart(...)

### Entrenamiento de KD contra temperatura

In [4]:
print(top_latex(data['students']['kd']['KD'],['Mejores resultados sobre logits segun diferencia con CE', 'Mejores resultados sobre logits segun diferencia con tutor'],drop=["D. Conv."]))


    \begin{minipage}[b]{.40\textwidth}

    \small
    \begin{tabular}{|l|l|l|l|r|r|r|}
        \hline
        \rowcolor[HTML]{9B9B9B} 
                          Modelo &             D. logits & $\Delta S$ \\ \hline

  \footnotesize ResNet18 &  \footnotesize KD, 10 &          -0.25 \\ \hline
 \footnotesize MobileNet &  \footnotesize KD, 10 &          -0.33 \\ \hline
  \footnotesize ResNet18 &   \footnotesize KD, 8 &          -0.43 \\ \hline
 \footnotesize MobileNet &   \footnotesize KD, 8 &          -0.52 \\ \hline
  \footnotesize ResNet18 &   \footnotesize KD, 5 &          -0.69 \\ \hline
        \end{tabular}
       \captionof{table}{Mejores resultados sobre logits segun diferencia con CE}
       \label{tab:forpol}
    \end{minipage}\qquad
    \begin{minipage}[b]{.40\textwidth}
    \hspace{1.5cm}
    \small
       \begin{tabular}{|l|l|l|l|r|r|r|}
        \hline
        \rowcolor[HTML]{9B9B9B} 
                         Modelo &                D. logits & $\Delta T$ \\ \hline

 \footn

El segundo experimento fue la destilación de la ResNet101 entrenada usando ambos modelos estudiantes, MobileNet y ResNet18. Se usó perdida KD y KD con CE probando distintos valores de temperatura en la perdida de destilación. 

De la observación de la tabla mejores segun cantidad de aprendizaje con respecto a la red tutora es posible notar que en los mejores casos se logra una diferencia bastante leve con respecto a lo que es capaz de de aprender el modelo por si solo desde los datos crudos usando CE. Viendo el grafico de accuracy con respecto a la red tutora es posible notar que ademas de esto, los casos que usan ResNet18 suelen obtener valores mas altos que aquellos que usan MobileNet. Ademmás, en general el uso de CE junto con KD no beneficia mucho el resultado final. Sobre la temperatura, es posible notar que dentro de los valores bajos el rango no varía mucho, pareciendo ser que los casos mas cercanos a 10 son los que presentan mejores resultados. Interesantemente, una temperatura mas alta parece mejorar el ratio entre entrenamiento y validación, sin embargo esto parece relacionarse mas la perdida sufrida en el accuracy de test que con una mejora general del algoritmo.

In [5]:
g=alt.hconcat(

grafico_con_barra_xy_CE(source=data['students']['kd']['KD'],
title="Diferencia de Acc. con ResNet101 contra temperatura",
y="delta_teacher",
x="temp",
color='student',
shape='log_dist')
    ,
grafico_con_barra_xy_CE(source=data['students']['kd']['KD'],
title="Ratio de Acc. en validacion y entrenamiento contra temperatura",
y="test/train",
x="temp",
color='student',
shape='log_dist'))    
g

alt.HConcatChart(...)

De la observacion del segundo par de graficos salta a la vista en primer lugar que en el caso de las diferencia de accuracy del modelo con respecto a su base los valores para ambos modelos son bastante cercanos. Además, una vez mas el uso de CE dentro de la perdida parece no afectar positivamente el valor dentro de la zona de mejor funcionamiento. Con respecto al tiempo de convergencia, no esposible ver que ningun hiperparametro del experimento, ni el modelo, ni la perdida ni la temperatura afecte positivamente.

In [6]:
g=alt.hconcat(
grafico_con_barra_xy_CE(source=data['students']['kd']['KD'],y='delta_student',bs=False,
title="Diferencia de Acc. en CE contra temperatura.",),
grafico_con_barra_xy_CE(source=data['students']['kd']['KD'],y='acc_conv',bs=False,
title="Tiempo de convergencia contra temperatura.",))
g

alt.HConcatChart(...)

### Destilacion usando features

Una vez fijado el valor de temperatura en 8, se procedió a probar los distintos tipos de destilaciones de features implementados.

Lo primero que salta a la vista es que al contrario de KD, donde todos los casos estudiados obtienen menos incremento contra la base que pone el modelo estudiante entrenado en CE, en este hay 8 casos que lograron superarse, sin alcanzar a superar a la red tutora. Interesantemente el incremento contra su propia base experimentado no depende tanto del modelo, por el contrario, influye mas el tipo de destilacion en logits usada y la combinacion destilacion de features y bloque de destilacion utilizado. Dentro de estos ultimos es posible notar que Hints funciona sostenidamente mal en las ultimas capas t sorprendentemente bien en la 2a, al igual que NST polinomial. Por el contrario, los mapas de atencion y NST lineal funcionan mejor en las ultimas capas, mientras que PKT funciona de manera regular en todos los casos. 
En general, el uso de KD solo en logits funciona mejor que su combinacion con CE o el uso solo de esta ultima. Punto aparte, al parecer mas importante aun que el tipo de destilacion usada es el tipo arquitectura. Al ver la diferencia de accuracy obtenido con respecto a la red tutora, es notorio que ResNet18 funciona mejor en todos los casos que MobileNet, cosa que se nota mas aun al mirar el ratio de entrenamiento y validacion, donde los unicos casos en que este empeoró fue en los que usaban MobileNet. Interesantemente, estos presentan tambien un tiempo menor para alcanzar el accuracy máximo obtenido, el cual de todas maneras es varios ordenes de magnitud mas bajo que el del resto de los casos.

In [7]:
source=pd.concat([normal_whole_source('KD'),normal_whole_source('KD_CE'),data['students']['feat']['CE']])#,normal_whole_source('CE')
#print(top_latex(source,['Mejores en \\features segun diferencia con CE', 'Mejores en \\features segun diferencia con tutor'],
#                drop=[]))
get_top(source,add_fields="delta_student delta_teacher 	test/train ".split(),
        drop_fields="train_step 	test_step 	student 	log_dist 	feat_dist 	layer".split())

,temp,delta_teacher,delta_student,test/train
9,8,-3.32,0.68,0.942603
17,8,-3.45,0.55,0.942938
38,8,-0.18,0.42,0.945123
15,8,-3.61,0.39,0.952848
7,8,-3.74,0.26,0.945691


In [8]:
   
def grafico_con_barra_y_CE(source=data['students']['kd']['KD'],
                        title="Diferencia de accuracy en entrenamiento con respecto a Resnet101.",
                        y="delta_teacher",
                        x="feat_dist",
                        fill='layer',
                        color='student',
                        shape='layer',
                        bs=True,
                        scale='linear'):
    
    
    if x=='dists':
        source=source.copy()
        source['dists']=source['feat_dist']+", "+source['log_dist']
    elif x=='feat,block':
        source=source.copy()
        source['feat,block']=[i+", "+str(j) for i,j in list(zip(source['feat_dist'],source['layer']))]
        

    
    
        
    chart=alt.Chart(source,title=title).mark_point(size=100).encode(
           
            y=alt.Y(y, type='quantitative',scale=alt.Scale(zero=True,base=2,constant=1,type=scale),
                    title=global_titles[y] if scale=='linear' else global_titles[y]+" [%s]"%scale),
            x=alt.X(x, title=global_titles[x]),
            color=alt.Color(color, legend=alt.Legend(title=global_titles[color])),
            fill=alt.Fill('%s:O'%fill, legend=alt.Legend(title=global_titles[fill]), 
                          scale=alt.Scale(scheme='pastel1')),
            shape=alt.Shape("%s:O"%shape,legend=alt.Legend(title=global_titles[shape])),
            opacity=alt.value(0.5)
        )


    bar=alt.Chart(bar_source).mark_rule(opacity=0.5).encode(
                        y=y,
                        color=alt.Color('model', legend=alt.Legend(title="Modelo")),
                        stroke=alt.Stroke('model', legend=alt.Legend(title="Modelo en Cross Entropy")),
                        size=alt.value(2))

    if bs:
        bar=alt.Chart(bar_source).mark_rule(opacity=0.5).encode(
                            y=y,
                            color=alt.Color('model', legend=alt.Legend(title="Modelo")),
                            stroke=alt.Stroke('model', legend=alt.Legend(title="Modelo en Cross Entropy")),
                            size=alt.value(2))
        d=bar+chart
        d.properties(width=600,height=600).configure_axis(titleFontSize=12).configure_title(fontSize=15)

        return d
    
    chart.properties(width=600,height=600).configure_axis(titleFontSize=12).configure_title(fontSize=15)
    return chart
    


In [9]:
g=alt.hconcat(grafico_con_barra_y_CE(source=source,
                                     title="Diferencia de Acc. con ResNet101 ",
                                       x='feat,block',
                                       y='delta_teacher',
                                       bs=True,
                                      shape='log_dist',scale='sqrt',
                                      ).properties(width=500,height=400).interactive()
              ,
grafico_con_barra_y_CE(source=source,
                       title="Ratio de Acc. en validacion y entrenamiento",
                       x='feat,block',
                       y='test/train',
                       bs=True,
                      shape='log_dist',scale='log'
                      ).properties(width=500,height=400).interactive()
)
g

alt.HConcatChart(...)

In [10]:
g=alt.hconcat(
grafico_con_barra_y_CE(source=source,
                       x='feat,block',
                       y='delta_student',
                       title="Diferencia de Acc. en CE.",
                       bs=True,
                      shape='log_dist',scale='sqrt',
                      ).properties(width=500,height=400).interactive(),

grafico_con_barra_y_CE(source=source[source['acc_conv']<50],
                       x='feat,block',
                       y='acc_conv',
                       title="Tiempo de convergencia.",
                       bs=False,
                      shape='log_dist',scale='linear'
                      ).properties(width=500,height=400).interactive()
)
g

alt.HConcatChart(...)

In [11]:

chart=alt.Chart(source[source['acc_conv']<50],title="Diferencia de accuracy en entrenamiento con respecto a Resnet101.").mark_point(size=150).encode(
    shape=alt.Shape('layer:O',legend=alt.Legend(title="Bloque"), scale=alt.Scale(scheme='darkgreen')),
    y=alt.Y("delta_teacher:Q",title="Delta accuracy.",scale=alt.Scale(zero=False,base=10,type='linear')),
    x=alt.X('acc_conv:Q',type='quantitative',title="Delta accuracy.",scale=alt.Scale(zero=False,base=10,type='linear')),
    color=alt.Fill('student', legend=alt.Legend(title="Modelo")),
    fill=alt.Color('layer:O', legend=alt.Legend(title="Bloque"), scale=alt.Scale(scheme='darkgreen')),
).interactive()
chart

alt.Chart(...)

## Datasets sintéticos GAN / VAE / Ruido

A diferencia de el aprendizaje supervisado regular, donde toda la información disponible para el aprendizaje proviene de los datos mismos, en destilación ademas del dataset de entrada, se dispone del conocimiento latente que contiene la red ya entrenada. La tarea cambia entonces desde aprender desde una base de datos a aprender desde las características que parametriza la salida o los mapas de caracteristicas de la red. Una diferencia importante con respecto al caso original es que este permite usar el espacio de entrada fuera de los puntos que contiene el dataset, cosa que podria permitir por ejemplo hacer posible un entrenamiento sin necesidad de usar el dataset, cosa que podría ser ampliamente provechosa en casos en que este no se encuentre disponible.


Al respecto de esto, se probó tres casos de datasets sintéticos, uno basado en auxiliary classifier GAN, uno basado en VAE y uno consistente de ruido gaussiano de varianza 1. Solo en el caso de auxiliary classifier GAN si se dispone de una etiqueta al momento de generar la muestra. Como perdida de ultima capa solo se usa KD de temperatura 8, en las features se usó distintos tipos de destilacion sobre distintas capas. Los valores de interes son los mismos que en el caso del dataset sintético.

In [12]:
source=pd.concat([data['students']['feat']['noise'],
                  data['students']['kd']['noise']])

source=source[source['noise']==1.0]

source=source.copy()
source['dataset']="Noise 1.0"

sourceKD=normal_whole_source('KD')
sourceKD=sourceKD[sourceKD['feat_dist']!='nst_poly']

source=pd.concat([source,
                 data['students']['kd']['VAE'],
                 data['students']['feat']['VAE'],
                 data['students']['kd']['GAN'],
                 data['students']['feat']['GAN'],
                 sourceKD
                 ])


print(top_latex(source,['Mejores en \\features segun diferencia con CE', 'Mejores en \\features segun diferencia con tutor'],
                drop=['D. logits'],keep=['dataset'],n_cases=10))


    \begin{minipage}[b]{.40\textwidth}

    \small
    \begin{tabular}{|l|l|l|l|r|r|r|}
        \hline
        \rowcolor[HTML]{9B9B9B} 
                          Modelo &                     D. Conv. &  dataset & $\Delta S$ \\ \hline

 \footnotesize MobileNet &        \footnotesize hint, 1 &  cifar10 &           0.68 \\ \hline
 \footnotesize MobileNet &        \footnotesize hint, 1 &      GAN &           0.54 \\ \hline
  \footnotesize ResNet18 &  \footnotesize nst\_linear, 2 &  cifar10 &           0.42 \\ \hline
 \footnotesize MobileNet &  \footnotesize nst\_linear, 3 &  cifar10 &           0.39 \\ \hline
 \footnotesize MobileNet &    \footnotesize att\_mean, 3 &  cifar10 &           0.26 \\ \hline
  \footnotesize ResNet18 &    \footnotesize att\_mean, 3 &  cifar10 &           0.13 \\ \hline
  \footnotesize ResNet18 &  \footnotesize nst\_linear, 2 &      GAN &           0.12 \\ \hline
  \footnotesize ResNet18 &         \footnotesize PKT, 1 &  cifar10 &           0.07 \\ \hline
  \foot

In [13]:
get_top(source,by='delta_teacher',add_fields=['dataset'],n_cases=10)

,dataset,student,log_dist,temp,feat_dist,layer,delta_teacher,delta_student
38,cifar10,ResNet18,KD,8,nst_linear,2,-0.18,0.42
31,cifar10,ResNet18,KD,8,att_mean,3,-0.47,0.13
34,GAN,ResNet18,KD,8,nst_linear,2,-0.48,0.12
45,cifar10,ResNet18,KD,8,PKT,1,-0.53,0.07
33,cifar10,ResNet18,KD,8,hint,1,-0.55,0.05
22,GAN,ResNet18,KD,8,att_max,2,-0.61,-0.01
27,GAN,ResNet18,KD,8,att_mean,3,-0.65,-0.05
27,cifar10,ResNet18,KD,8,att_max,3,-0.65,-0.05
26,cifar10,ResNet18,KD,8,att_max,2,-0.67,-0.07
32,GAN,ResNet18,KD,8,nst_linear,0,-0.67,-0.07


In [14]:
g=alt.hconcat(grafico_con_barra_y_CE(source=source,
                                     title="Diferencia de Acc. con ResNet101 ",
                                       x='feat,block',
                                       y='delta_teacher',
                                       bs=True,
                                      shape='dataset',scale='sqrt',
                                      ).properties(width=500,height=400).interactive()
              ,
grafico_con_barra_y_CE(source=source,
                       title="Ratio de Acc. en validacion y entrenamiento",
                       x='feat,block',
                       y='test/train',
                       bs=True,
                      shape='dataset',scale='log'
                      ).properties(width=500,height=400).interactive()
)
g

alt.HConcatChart(...)

In [15]:
g=alt.hconcat(
grafico_con_barra_y_CE(source=source,
                       x='feat,block',
                       y='delta_student',
                       title="Diferencia de Acc. en CE.",
                       bs=True,
                      shape='dataset',scale='sqrt',
                      ).properties(width=500,height=400).interactive(),

grafico_con_barra_y_CE(source=source[source['acc_conv']<50],
                       x='feat,block',
                       y='acc_conv',
                       title="Tiempo de convergencia.",
                       bs=False,
                      shape='dataset',scale='linear'
                      ).properties(width=500,height=400).interactive()
)
g

alt.HConcatChart(...)

## Graficos Noise

El ultimo experimiento realizado fue la destilación sobre un dataset con distintos niveles de corrupción. Para esto se entrenó añadiendo ruido gaussiano de potencia 0 a 1 sobre el dataset cifar10, destilando con perdida KD en los logits y distintos tipos de destilacion de features.

Para estudiar el efecto del ruido se obtuvo las curvas de decaimiento para el accuracy en test y el ratio entre los accuracys de validacion y entrenamiento.

In [16]:
source=pd.concat([data['students']['feat']['noise'],data['students']['kd']['noise'],normal_whole_source('KD')])
#source['noise']=[float(i.split(',')[1]) for i in source['transform']]



In [17]:
g=alt.hconcat(
    noise_plot(source=source,
               title="Diferencia con ResNet101 contra ruido en ResNet18",
               y='delta_teacher',
               color='feat_dist',
               shape='layer',
               student='ResNet18').interactive()
    ,
    noise_plot(source=source,
               title="Diferencia con ResNet101 contra ruido en MobileNet",
               y='delta_teacher',
               color='feat_dist',
               shape='layer',
               student='MobileNet').interactive()
)
g

alt.HConcatChart(...)

In [18]:
g=alt.hconcat(
    noise_plot(source=source,
               title="Diferencia con ResNet101 contra ruido en ResNet18",
               y='delta_teacher',
               color='layer',
               shape='feat_dist',
               student='ResNet18').interactive()
    ,
    noise_plot(source=source,
               title="Diferencia con ResNet101 contra ruido en MobileNet",
               y='delta_teacher',
               color='layer',
               shape='feat_dist',
               student='MobileNet').interactive()
) 
g

alt.HConcatChart(...)

# Imagenet

In [19]:
drop_cols = ['Unnamed: 0',  'lr', 'resume', 'epochs', 'train_batch_size',
             'test_batch_size', 'batch_size', 'teacher', 
             'pre', 'test_eval', 'train_eval', 'test_teacher/acc',
             'train_teacher/acc']
source=pd.read_csv("Imagenet/summary.csv")
cols = [i for i in source.columns if i in drop_cols]

#source.index = pd.Index(["Imagenet/"+ "/".join([str(i) for i in [row['exp_name'],row["student"], row["distillation"],row['layer'] ]] ) 
#                     for i, row in source.iterrows()])

source['test/train']=source['test_acc']/source["train_acc"]
source['delta_teacher']=source['test_acc']/source['test_teacher/acc']
# Falta el modelo estudiante, obtener aparters
source=source.drop(columns=cols)
source
# 

,test_acc,test_loss,train_acc,train_loss,epoch,student,transform,dataset,exp_name,distillation,last_layer,student_layer,teacher_layer,layer,shape,test/train,delta_teacher
0,68.972638,0.000014,67.870418,0.000013,200,MobileNetV2,"none,","ImageNet,",exp_feats,PKT,"KD,T-8",30.0,39.0,1.0,224.0,1.016240,0.808227
1,64.254001,0.001087,65.467217,0.000475,100,ResNet18,"none,","ImageNet,",exp_feats,nst_linear,"KD,T-8",17.0,39.0,1.0,224.0,0.981468,0.758255
2,70.366546,0.000014,71.732060,0.000010,200,ResNet18,"none,","ImageNet,",exp_feats,PKT,"KD,T-8",17.0,39.0,1.0,224.0,0.980964,0.830389
3,38.317997,0.000028,34.312226,0.000028,49,MobileNetV2,"none,","ImageNet,",post_wea,PKT,"KD,T-8",30.0,39.0,1.0,224.0,1.116745,0.506008
4,70.314920,0.000014,71.713991,0.000010,200,MobileNetV2,"none,","ImageNet,",test_1,"KD,T-8.0",NaN,NaN,NaN,NaN,NaN,0.980491,0.829779
5,62.168301,0.000029,57.927207,0.000017,50,ResNet18,"none,","ImageNet,",NaN,"KD,T-8.0",NaN,NaN,NaN,NaN,NaN,1.073214,0.733642
6,69.334022,0.000014,67.478059,0.000013,200,MobileNetV2,"none,","ImageNet,",test_mobilenet,"KD,T-8.0",NaN,NaN,NaN,NaN,NaN,1.027505,0.810794
7,59.148561,0.000013,51.509740,0.000014,100,MobileNetV2,"none,","ImageNet,",ultimors,PKT,"KD,T-8",30.0,39.0,1.0,224.0,1.148299,0.771545


In [20]:
source=source[source['epoch']>=50]
source=source[source['exp_name'] != 'test_1']
source=source.reset_index()
source

,index,test_acc,test_loss,train_acc,train_loss,epoch,student,transform,dataset,exp_name,distillation,last_layer,student_layer,teacher_layer,layer,shape,test/train,delta_teacher
0,0,68.972638,0.000014,67.870418,0.000013,200,MobileNetV2,"none,","ImageNet,",exp_feats,PKT,"KD,T-8",30.0,39.0,1.0,224.0,1.016240,0.808227
1,1,64.254001,0.001087,65.467217,0.000475,100,ResNet18,"none,","ImageNet,",exp_feats,nst_linear,"KD,T-8",17.0,39.0,1.0,224.0,0.981468,0.758255
2,2,70.366546,0.000014,71.732060,0.000010,200,ResNet18,"none,","ImageNet,",exp_feats,PKT,"KD,T-8",17.0,39.0,1.0,224.0,0.980964,0.830389
3,5,62.168301,0.000029,57.927207,0.000017,50,ResNet18,"none,","ImageNet,",NaN,"KD,T-8.0",NaN,NaN,NaN,NaN,NaN,1.073214,0.733642
4,6,69.334022,0.000014,67.478059,0.000013,200,MobileNetV2,"none,","ImageNet,",test_mobilenet,"KD,T-8.0",NaN,NaN,NaN,NaN,NaN,1.027505,0.810794
5,7,59.148561,0.000013,51.509740,0.000014,100,MobileNetV2,"none,","ImageNet,",ultimors,PKT,"KD,T-8",30.0,39.0,1.0,224.0,1.148299,0.771545


In [21]:
epoch_data=json.load(open('Imagenet/epoch_summary.json','r'))
for k in epoch_data.keys():
    print(k)

Imagenet/exp_feats/ResNet152/MobileNetV2/PKT/1//
Imagenet/exp_feats/ResNet152/ResNet18/Imagenet/nst_linear/1//
Imagenet/exp_feats/ResNet152/ResNet18/Imagenet/PKT/1//
Imagenet/test_mobilenet/KD/T-8.0//
Imagenet/test_1/KD/T-8.0//
Imagenet/test_5/KD/T-8.0//
Imagenet/ultimors/ResNet152/MobileNetV2/PKT/1//


In [22]:

#da['delta_teacher']=da['test_acc']-data["teachers"].at['ResNet101','test_acc']
#da['delta_student']=[r['test_acc']-data['teachers'].at[r['student'],'test_acc'] for i,r in da.iterrows()]
source['time_data']=["Imagenet/exp_feats/ResNet152/MobileNetV2/PKT/1//",
    "Imagenet/exp_feats/ResNet152/ResNet18/Imagenet/nst_linear/1//",
    "Imagenet/exp_feats/ResNet152/ResNet18/Imagenet/PKT/1//",
    #"Imagenet/test_1/KD/T-8.0//",
    "Imagenet/test_5/KD/T-8.0//",
    "Imagenet/test_mobilenet/KD/T-8.0//",
    #
    ]

#print(to_latex(source))

ValueError: Length of values does not match length of index

In [ ]:
epoch_data=json.load(open('Imagenet/epoch_summary.json','r'))

def info_df(row,dic):
    df=pd.DataFrame.from_dict(dic[row['time_data']])
    l=len(df)
    print(row['time_data'],l)
    p = pd.DataFrame.from_dict({
    "epoch":range(l),
    "key":[", ".join([row["student"],row["distillation"]]) for i in range(l)],
    "total_epochs":[row["epoch"] for i in range(l)],
    "student":[row["student"] for i in range(l)],
    "dataset":[row["dataset"] for i in range(l)],
    "exp_name":[row["exp_name"] for i in range(l)],
    "distillation":[row["distillation"] for i in range(l)],
    "time_data":[row["time_data"] for i in range(l)]}),
    
    return df.join(p)



#time_data=[pd.DataFrame.from_dict(epoch_data[i]) for i in source['time_data']]
time_data=pd.concat([info_df(row,epoch_data) for i,row in source.iterrows()])

time_data.head()

In [ ]:
time_data.columns

In [ ]:
def plot_train(source,
    y="test/acc",
    x='epoch',#'test/wall_time'#
    #y_scale=alt.Scale(zero=False,base=2,type='linear')
    color='key'):
    
    cols=['train/acc', 'train/loss', 'train/eval', 'train/wall_time', 'test/acc',
       'test/loss', 'test/eval', 'test/wall_time', 'epoch', 'key',
       'total_epochs',  'dataset', 'exp_name', 'distillation',
       'time_data']
    
    cols=[i for i in cols if i not in [y,x,color]]
    #print(cols)
    source=source.copy().drop(columns=cols)


    # Create a selection that chooses the nearest point & selects based on x-value
    nearest = alt.selection(type='single', nearest=True, on='mouseover',
                            fields=[x], empty='none')

    y+=":Q"
    x+=":Q"
    color+=":N"
    
    y_scaled=alt.Y(y, type='quantitative',
                   title="acc de test",
                   scale=alt.Scale(zero=False,base=2,exponent=2,type='pow')
    )
    # The basic line
    line = alt.Chart(source).mark_line(interpolate='basis').encode(
        x=x,
        y=y_scaled,
        color=color,
        opacity=alt.value(0.6)
    )

    # Transparent selectors across the chart. This is what tells us
    # the x-value of the cursor
    selectors = alt.Chart(source).mark_point().encode(
        x=x,
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    # Draw points on the line, and highlight based on selection
    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
        shape='student'
    )

    # Draw text labels near the points, and highlight based on selection
    text = line.mark_text(align='left', dx=5, dy=-5,size=10,fontWeight='bold',
                          blend='difference',line=True).encode(
        text=alt.condition(nearest, y, alt.value(' ')),
        opacity=alt.value(1)
    )

    # Draw a rule at the location of the selection
    rules = alt.Chart(source).mark_rule(color='gray').encode(
        x=x,
    ).transform_filter(
        nearest
    )

    # Put the five layers into a chart and bind the data
    #return source
    return alt.layer(
        line, selectors, points, rules, text
    ).properties(
        width=600, height=500
    ).interactive()
    

In [ ]:
    
g=plot_train(time_data)
g
save(g,'exp_feats/Imagenet.png')

In [ ]:
g=alt.hconcat(
grafico_con_barra_xy_CE(source=source,y='delta_student',bs=False,
title="Diferencia de Acc. en CE contra temperatura.",),
grafico_con_barra_xy_CE(source=source,y='acc_conv',bs=False,
title="Tiempo de convergencia contra temperatura.",))
g

In [ ]:
j=[info_df(row,epoch_data) for i,row in source.iterrows()]

In [ ]:
source=time_data[time_data['total_epochs']==200]
source

In [ ]:
 En primer lugar, es posible notar que mediante destilación no es posible aprender etiquetas aleatorias, tal y como hace notar la ausencia de un crecimiento muy grande en el \textit{overfitting} en el gráfico del ratio entre accuracy en validación y entrenamiento y el hecho de que muchos casos entrenados sobre ruido dejaron de aprender a tan temprana época.
Esto da indicios de que en destilación es bastante importante la vecindad sobre la que se trabaja, es decir, no es posible transferir el conocimiento de una red a otra sobre todo el espacio de entrada, pero, si se puede de manera bastante satisfactoria sobre el espacio de las imágenes similares a la base de datos de entrenamiento donde se entrenó la red tutor. 
En segundo lugar, la información que la transferencia de \features da, compensa la información que se pierde al usar imágenes no reales, cosa que se puede notar al comparar el desempeño sobre datos con colores similares pero sin forma como son los generados con VAE con los datos con formas similares que genera GAN y al notar el mal comportamiento en todos los casos que tiene la destilación sin features. 
dd

In [ ]:
 En primer lugar, es posible notar que mediante destilación no es posible aprender etiquetas aleatorias, tal y como hace notar la ausencia de un crecimiento muy grande en el \textit{overfitting} en el gráfico del ratio entre accuracy en validación y entrenamiento y el hecho de que muchos casos entrenados sobre ruido dejaron de aprender a tan temprana época.
Esto da indicios de que en destilación es bastante importante la vecindad sobre la que se trabaja, es decir, no es posible transferir el conocimiento de una red a otra sobre todo el espacio de entrada, pero, si se puede de manera bastante satisfactoria sobre el espacio de las imágenes similares a la base de datos de entrenamiento donde se entrenó la red tutor. 
En segundo lugar, la información que la transferencia de \features da, compensa la información que se pierde al usar imágenes no reales, cosa que se puede notar al comparar el desempeño sobre datos con colores similares pero sin forma como son los generados con VAE con los datos con formas similares que genera GAN y al notar el mal comportamiento en todos los casos que tiene la destilación sin features. 
dd

In [ ]:
j=[info_df(row,epoch_data) for i,row in source.iterrows()]

In [ ]:
source=time_data[time_data['total_epochs']==200]
source

In [ ]:
 En primer lugar, es posible notar que mediante destilación no es posible aprender etiquetas aleatorias, tal y como hace notar la ausencia de un crecimiento muy grande en el \textit{overfitting} en el gráfico del ratio entre accuracy en validación y entrenamiento y el hecho de que muchos casos entrenados sobre ruido dejaron de aprender a tan temprana época.
Esto da indicios de que en destilación es bastante importante la vecindad sobre la que se trabaja, es decir, no es posible transferir el conocimiento de una red a otra sobre todo el espacio de entrada, pero, si se puede de manera bastante satisfactoria sobre el espacio de las imágenes similares a la base de datos de entrenamiento donde se entrenó la red tutor. 
En segundo lugar, la información que la transferencia de \features da, compensa la información que se pierde al usar imágenes no reales, cosa que se puede notar al comparar el desempeño sobre datos con colores similares pero sin forma como son los generados con VAE con los datos con formas similares que genera GAN y al notar el mal comportamiento en todos los casos que tiene la destilación sin features. 
dd